In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import os

# Getting Pokemon and their Stats

In [2]:
page = requests.get('https://www.serebii.net/pokemon/all.shtml')

In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [4]:
table = soup.find(class_='dextable')

In [5]:
heads = table.find_all('td', class_='fooevo')
heads = [i.text for i in heads]
heads = [re.sub('[\t|\r|\n]','',i) for i in heads]
heads.remove('Base Stats')
heads.remove('Pic')
heads

['No.',
 'Name',
 'Type',
 'Abilities',
 'HP',
 'Att',
 'Def',
 'S.Att',
 'S.Def',
 'Spd']

In [10]:
x = table.find_all('tr')[2:]
poke = [i.text.split('\n') for i in x]
sub = '[\t|\r|\n]'
poke = [[re.sub(sub,'',i) for i in j if len(re.sub(sub,'',i))>=1 and i != ' '] for j in poke if len(j)>1]

rows = table.find_all('tr')
all_links = [i.find_all('a') for i in rows]
all_links = [i for i in all_links if len(i)>0]
all_links = ['/'.join([i.get('href') for i in a]) for a in all_links]
all_types = [re.findall('/type/(\w+)',i) for i in all_links if len(re.findall('/type/(\w+)',i)) >= 1]
all_types

pokemon = pd.DataFrame({
    heads[0]:[i[0] for i in poke],
    heads[1]:[i[1] for i in poke],
    heads[2]:all_types,
    heads[3]:[i[2] for i in poke],
    f"Base_{heads[4]}":[i[3] for i in poke],
    f"Base_{heads[5]}":[i[4] for i in poke],
    f"Base_{heads[6]}":[i[5] for i in poke],
    f"Base_{heads[7]}":[i[6] for i in poke],
    f"Base_{heads[8]}":[i[7] for i in poke],
    f"Base_{heads[9]}":[i[8] for i in poke]
})

In [11]:
pokemon.head()

,No.,Name,Type,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd
0,#001,Bulbasaur,"[grass, poison]",Overgrow Chlorophyll,45,49,49,65,65,45
1,#002,Ivysaur,"[grass, poison]",Overgrow Chlorophyll,60,62,63,80,80,60
2,#003,Venusaur,"[grass, poison]",Overgrow Chlorophyll,80,82,83,100,100,80
3,#004,Charmander,[fire],Blaze Solar Power,39,52,43,60,50,65
4,#005,Charmeleon,[fire],Blaze Solar Power,58,64,58,80,65,80


In [12]:
pokemon['Type'] = pokemon['Type'].apply(lambda x: [i.capitalize() for i in x])
pokemon['No.'] = pd.to_numeric(pokemon['No.'].str.replace('#',''))

In [13]:
pokemon.head()

,No.,Name,Type,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd
0,1,Bulbasaur,"[Grass, Poison]",Overgrow Chlorophyll,45,49,49,65,65,45
1,2,Ivysaur,"[Grass, Poison]",Overgrow Chlorophyll,60,62,63,80,80,60
2,3,Venusaur,"[Grass, Poison]",Overgrow Chlorophyll,80,82,83,100,100,80
3,4,Charmander,[Fire],Blaze Solar Power,39,52,43,60,50,65
4,5,Charmeleon,[Fire],Blaze Solar Power,58,64,58,80,65,80


In [14]:
pokemon['Type1'] = [i[0] for i in pokemon['Type']]
pokemon['Type2'] = [i[1] if len(i)==2 else float('NaN') for i in pokemon['Type']]
pokemon = pokemon.drop(columns=['Type'])
pokemon.head()

,No.,Name,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd,Type1,Type2
0,1,Bulbasaur,Overgrow Chlorophyll,45,49,49,65,65,45,Grass,Poison
1,2,Ivysaur,Overgrow Chlorophyll,60,62,63,80,80,60,Grass,Poison
2,3,Venusaur,Overgrow Chlorophyll,80,82,83,100,100,80,Grass,Poison
3,4,Charmander,Blaze Solar Power,39,52,43,60,50,65,Fire,NaN
4,5,Charmeleon,Blaze Solar Power,58,64,58,80,65,80,Fire,NaN


# Getting Pokemon Abilities
Will modify the dataframe above with the data scraped here before saving both

In [15]:
ability_url = 'https://pokemondb.net/ability'
ability_page = requests.get(ability_url)
soup = BeautifulSoup(ability_page.content, 'html.parser')


In [16]:
headers = [i.text for i in rows[0].find_all('th')]
headers

[]

In [17]:
#removing extra empty list items with less code
ab_table = soup.find('table', id='abilities')
rows = ab_table.find_all('tr')
abilities = [i.text.split('\n')[1:-1] for i in rows[1:]]
abilities

[['Adaptability', '13', 'Powers up moves of the same type.', '4'],
 ['Aerilate', '2', 'Turns Normal-type moves into Flying-type moves.', '6'],
 ['Aftermath', '8', 'Damages the attacker landing the finishing hit.', '4'],
 ['Air Lock', '1', 'Eliminates the effects of weather.', '3'],
 ['Analytic', '12', 'Boosts move power when the Pokémon moves last.', '5'],
 ['Anger Point', '9', 'Maxes Attack after taking a critical hit.', '4'],
 ['Anticipation', '14', "Senses a foe's dangerous moves.", '4'],
 ['Arena Trap', '3', 'Prevents the foe from fleeing.', '3'],
 ['Aroma Veil',
  '4',
  'Protects allies from attacks that limit their move choices.',
  '6'],
 ['As One', '2', 'Combines Unnerve and Chilling Neigh/Grim Neigh', '8'],
 ['Aura Break', '2', 'Reduces power of Dark- and Fairy-type moves.', '6'],
 ['Bad Dreams', '1', "Reduces a sleeping foe's HP.", '4'],
 ['Ball Fetch', '1', 'Retrieves a Poké Ball from a failed throw.', '8'],
 ['Battery', '1', "Raises power of teammates' Special moves.", '7'

Ignoring the Count of pokemon columns

In [18]:
ability_df = pd.DataFrame({
    'Ability':[i[0] for i in abilities],
    'Description':[i[2] for i in abilities],
    'Gen Introduced':[i[3] for i in abilities]
})
ability_df

,Ability,Description,Gen Introduced
0,Adaptability,Powers up moves of the same type.,4
1,Aerilate,Turns Normal-type moves into Flying-type moves.,6
2,Aftermath,Damages the attacker landing the finishing hit.,4
3,Air Lock,Eliminates the effects of weather.,3
4,Analytic,Boosts move power when the Pokémon moves last.,5
...,...,...,...
261,White Smoke,Prevents other Pokémon from lowering its stats.,3
262,Wimp Out,Switches out when HP drops below half.,7
263,Wonder Guard,Only supereffective moves will hit.,3
264,Wonder Skin,Makes status-changing moves more likely to miss.,5


In [19]:
pokemon.head()

,No.,Name,Abilities,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd,Type1,Type2
0,1,Bulbasaur,Overgrow Chlorophyll,45,49,49,65,65,45,Grass,Poison
1,2,Ivysaur,Overgrow Chlorophyll,60,62,63,80,80,60,Grass,Poison
2,3,Venusaur,Overgrow Chlorophyll,80,82,83,100,100,80,Grass,Poison
3,4,Charmander,Blaze Solar Power,39,52,43,60,50,65,Fire,NaN
4,5,Charmeleon,Blaze Solar Power,58,64,58,80,65,80,Fire,NaN


In [20]:
from itertools import product, permutations

In [21]:
all_abilities = [i for i in ability_df['Ability']]
ability_sep = []
for i in pokemon['Abilities']:
    no1 = i.split(' ')
    no2 = [' '.join(f) for f in permutations(i.split(), r=2)]
    no3 = [' '.join(f) for f in permutations(i.split(), r=3)]
    
    mon = []
    for j in no1:
        if j in all_abilities and j not in mon:
            mon.append(j)
    for j in no2:
        if j in all_abilities and j not in mon:
            mon.append(j)
    for j in no3:
        if j in all_abilities and j not in mon:
            mon.append(j)
            
    ability_sep.append(mon)

In [22]:
ability_sep

[['Overgrow', 'Chlorophyll'],
 ['Overgrow', 'Chlorophyll'],
 ['Overgrow', 'Chlorophyll'],
 ['Blaze', 'Solar Power'],
 ['Blaze', 'Solar Power'],
 ['Blaze', 'Solar Power'],
 ['Torrent', 'Rain Dish'],
 ['Torrent', 'Rain Dish'],
 ['Torrent', 'Rain Dish'],
 ['Shield Dust', 'Run Away'],
 ['Shed Skin'],
 ['Tinted Lens'],
 ['Shield Dust', 'Run Away'],
 ['Shed Skin'],
 ['Swarm', 'Sniper'],
 ['Keen Eye', 'Tangled Feet', 'Big Pecks'],
 ['Keen Eye', 'Tangled Feet', 'Big Pecks'],
 ['Keen Eye', 'Tangled Feet', 'Big Pecks'],
 ['Guts', 'Hustle', 'Run Away'],
 ['Guts', 'Hustle', 'Run Away'],
 ['Sniper', 'Keen Eye'],
 ['Sniper', 'Keen Eye'],
 ['Intimidate', 'Unnerve', 'Shed Skin'],
 ['Intimidate', 'Unnerve', 'Shed Skin'],
 ['Static', 'Lightning Rod'],
 ['Static', 'Lightning Rod'],
 ['Sand Veil', 'Sand Rush'],
 ['Sand Veil', 'Sand Rush'],
 ['Rivalry', 'Hustle', 'Poison Point'],
 ['Rivalry', 'Hustle', 'Poison Point'],
 ['Rivalry', 'Poison Point', 'Sheer Force'],
 ['Rivalry', 'Hustle', 'Poison Point'],
 ['

In [23]:
pokemon = pokemon.drop(columns=['Abilities'])
for i in range(max([len(i) for i in ability_sep])):
    col = f"Ability{i+1}"
    pokemon[col] = [j[i] if len(j) > i else float('NaN') for j in ability_sep]
pokemon.head()

,No.,Name,Base_HP,Base_Att,Base_Def,Base_S.Att,Base_S.Def,Base_Spd,Type1,Type2,Ability1,Ability2,Ability3,Ability4
0,1,Bulbasaur,45,49,49,65,65,45,Grass,Poison,Overgrow,Chlorophyll,NaN,NaN
1,2,Ivysaur,60,62,63,80,80,60,Grass,Poison,Overgrow,Chlorophyll,NaN,NaN
2,3,Venusaur,80,82,83,100,100,80,Grass,Poison,Overgrow,Chlorophyll,NaN,NaN
3,4,Charmander,39,52,43,60,50,65,Fire,NaN,Blaze,Solar Power,NaN,NaN
4,5,Charmeleon,58,64,58,80,65,80,Fire,NaN,Blaze,Solar Power,NaN,NaN


In [24]:
pokemon.to_csv('pokemon_data.csv', index=False)

In [283]:
ability_df.to_csv('ability_df.csv', index=False)

# Pokemon Moves
More complicated of a scrape

In [20]:
move_url = 'https://pokemondb.net/move/all'
move_page = requests.get(move_url)
soup = BeautifulSoup(move_page.content, 'html.parser')

In [21]:
move_table = soup.find('table', id='moves')
move_table

<table class="data-table block-wide" id="moves">
<thead>
<tr>
<th class="sorting" data-sort="string"><div class="sortwrap">Name</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Type</div></th> <th class="sorting" data-sort="string"><div class="sortwrap">Cat.</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Power</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Acc.</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">PP</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">TM</div></th> <th><div class="sortwrap">Effect</div></th> <th class="sorting" data-blanks="1" data-sort="int"><div class="sortwrap">Prob. (%)</div></th> </tr>
</thead>
<tbody>
<tr>
<td class="cell-name"><a class="ent-name" href="/move/10000000-volt-thunderbolt" title="View details for 10,000,000 Volt Thunderbolt">10,000,000 Volt Thunderbolt</a></td><td class="ce

In [22]:
move_rows = move_table.find_all('tr')
move_heads = [i.text for i in move_rows[0].find_all('th')]
move_names = [i.text for i in move_table.find_all('a',class_='ent-name')]
move_type = [i.text for i in move_table.find_all('td', class_='cell-icon') if i.text != '—' and len(i.text)>0]
move_cat = [''.join((i.get('data-sort-value'),i.text)) for i in move_table.find_all('td', class_='cell-icon text-center')]
move_numbers = [[i.text for i in x.find_all(class_='cell-num')] for x in move_table.find_all('tr')[1:]]
move_numbers = [['—' if i == '' else i for i in n] for n in move_numbers]
move_desc = [i.text for i in move_table.find_all('td',class_='cell-long-text')]
move_heads

['Name', 'Type', 'Cat.', 'Power', 'Acc.', 'PP', 'TM', 'Effect', 'Prob. (%)']

In [23]:
all_moves = pd.DataFrame({
    move_heads[0]:move_names,
    move_heads[1]:move_type,
    move_heads[2]:move_cat,
    move_heads[3]:[i[0] for i in move_numbers],
    move_heads[4]:[i[1] for i in move_numbers],
    move_heads[5]:[i[2] for i in move_numbers],
    move_heads[6]:[i[3] for i in move_numbers],
    'Prob(%)':[i[4] for i in move_numbers],
    move_heads[7]:move_desc

})
all_moves.head()

,Name,Type,Cat.,Power,Acc.,PP,TM,Prob(%),Effect
0,"10,000,000 Volt Thunderbolt",Electric,special,195,—,1,—,—,Pikachu-exclusive Z-Move. High critical hit ra...
1,Absorb,Grass,special,20,100,25,—,—,User recovers half the HP inflicted on opponent.
2,Accelerock,Rock,physical,40,100,20,—,—,User attacks first.
3,Acid,Poison,special,40,100,30,—,10,May lower opponent's Special Defense.
4,Acid Armor,Poison,status,—,—,20,—,—,Sharply raises user's Defense.


In [24]:
for i in ['Type','Power','Acc.','PP','TM','Effect','Prob(%)']:
    all_moves[i] = all_moves[i].str.replace('—', 'NaN')
all_moves.head()

,Name,Type,Cat.,Power,Acc.,PP,TM,Prob(%),Effect
0,"10,000,000 Volt Thunderbolt",Electric,special,195,NaN,1,NaN,NaN,Pikachu-exclusive Z-Move. High critical hit ra...
1,Absorb,Grass,special,20,100,25,NaN,NaN,User recovers half the HP inflicted on opponent.
2,Accelerock,Rock,physical,40,100,20,NaN,NaN,User attacks first.
3,Acid,Poison,special,40,100,30,NaN,10,May lower opponent's Special Defense.
4,Acid Armor,Poison,status,NaN,NaN,20,NaN,NaN,Sharply raises user's Defense.


In [301]:
all_moves.to_csv('all_pokemon_moves.csv', index=False)

# Downloading all Pokemon Sprites (128x128) png

In [169]:
try:
    os.mkdir('sprites')
except:
    pass

In [202]:
dexurl = 'https://pokemondb.net/pokedex/national'
dexpage = requests.get(dexurl).content
dexsoup = BeautifulSoup(dexpage, 'html.parser')

In [209]:
good_part = dexsoup.find(id='main')
dex = good_part.find_all(class_='img-fixed img-sprite')
for i in good_part.find_all(class_='img-fixed img-sprite img-sprite-v18'):
    dex.append(i)
dex

[<span class="img-fixed img-sprite" data-alt="Bulbasaur sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/bulbasaur.png"></span>,
 <span class="img-fixed img-sprite" data-alt="Ivysaur sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/ivysaur.png"></span>,
 <span class="img-fixed img-sprite" data-alt="Venusaur sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/venusaur.png"></span>,
 <span class="img-fixed img-sprite" data-alt="Charmander sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/charmander.png"></span>,
 <span class="img-fixed img-sprite" data-alt="Charmeleon sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/charmeleon.png"></span>,
 <span class="img-fixed img-sprite" data-alt="Charizard sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/charizard.png"></span>,
 <span class="img-fixed img-sprite" data-alt="Squirtle sprite" data-src="https://img.pokemondb.net/sprites/bank/normal/squirtle.png"></spa

In [210]:
len(good_part.find_all(class_='img-fixed img-sprite img-sprite-v18'))

89

In [211]:
sprites = [(i.get('data-alt'), i.get('data-src')) for i in dex]

In [212]:
sprites[:5]

[('Bulbasaur sprite',
  'https://img.pokemondb.net/sprites/bank/normal/bulbasaur.png'),
 ('Ivysaur sprite',
  'https://img.pokemondb.net/sprites/bank/normal/ivysaur.png'),
 ('Venusaur sprite',
  'https://img.pokemondb.net/sprites/bank/normal/venusaur.png'),
 ('Charmander sprite',
  'https://img.pokemondb.net/sprites/bank/normal/charmander.png'),
 ('Charmeleon sprite',
  'https://img.pokemondb.net/sprites/bank/normal/charmeleon.png')]

In [213]:
for i in sprites[-90:]:
    link = i[1]
    name = i[0].replace(' ','_')
    time.sleep(1)
    with open(f"sprites/{name}.png", 'wb') as img:
        img.write(requests.get(link).content)

# Getting Pokemon Character Details

In [25]:
for i in pokemon['Name']:
    print(i)

Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran♀
Nidorina
Nidoqueen
Nidoran♂
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetch'd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seaking

In [26]:
base_url = "https://www.serebii.net/pokedex-swsh/"

In [119]:
deet = requests.get("https://pokemondb.net/pokedex/toxtricity").content

In [120]:
deetsoup = BeautifulSoup(deet, 'html.parser')
deetsoup.find('div',class_='tabset-moves-game sv-tabs-wrapper')

<div class="tabset-moves-game sv-tabs-wrapper">
<div class="sv-tabs-tab-list">
<a class="sv-tabs-tab active" href="#tab-moves-18">Sword/Shield</a>
</div>
<div class="sv-tabs-panel-list">
<div class="sv-tabs-panel active" id="tab-moves-18">
<div class="grid-row"> <div class="grid-col span-lg-6"><h3>Moves learnt by level up</h3> <p class="text-small"><em>Toxtricity</em> learns the following moves in Pokémon Sword &amp; Shield at the levels specified.</p> <div class="tabset-moves-game-form sv-tabs-wrapper"> <div class="sv-tabs-tab-list"><a class="sv-tabs-tab text-small active" href="#tab-moves-18-level-849">Low Key Form</a><a class="sv-tabs-tab text-small" href="#tab-moves-18-level-11148">Amped Form</a> </div> <div class="sv-tabs-panel-list"><div class="sv-tabs-panel active" id="tab-moves-18-level-849"><div class="resp-scroll"><table class="data-table"><thead><tr><th class="sorting" data-sort="int"><div class="sortwrap">Lv.</div></th> <th class="sorting" data-sort="string"><div class="sor

In [121]:
deetsoup.find('img').get('src')

'https://img.pokemondb.net/artwork/toxtricity-low-key.jpg'

In [122]:
re.findall('h3>(.*)h3',str(deetsoup.find('div', class_='tabset-moves-game sv-tabs-wrapper')))

['Moves learnt by level up</',
 'Moves learnt on evolution</',
 'Egg moves</',
 'Move Tutor moves</',
 'Moves learnt by TM</',
 'Moves learnt by TR</']

# get_moves is broken, some characters have unique divs, looking for solution

In [123]:
def get_moves(soup):
    move_cats = [i.text for i in soup.find('div', class_='tabset-moves-game sv-tabs-wrapper').find_all('h3')]
    move_tables = soup.find(class_="sv-tabs-panel active").find_all(class_='data-table')

    output = {}
    all_h3s = []
    for i,j in enumerate(move_tables):
        category = move_cats[i]
        if category in output.keys():
            break
        else:
            output[category] = []
            
            moves = [[f.text for f in h.find_all('td') if f.text != ''] for h in j]
            if category.endswith('level up') or category.endswith('TM') or category.endswith('TR'):
                moves = [[f for f in h[:2]] for h in moves]
            else:
                moves = [h[0] for h in moves]
            
            
            for x in moves:
                output[category].append(x)
    return output

In [124]:
get_moves(deetsoup)

{}

In [33]:
# all move categories, some are duplicates for different games, will only pull the first one
deetsoup.find('div', class_='tabset-moves-game sv-tabs-wrapper').find_all('h3')


[<h3>Moves learnt by level up</h3>,
 <h3>Egg moves</h3>,
 <h3>Move Tutor moves</h3>,
 <h3>Moves learnt by TM</h3>,
 <h3>Moves learnt by TR</h3>]

In [34]:
# all move tables
[[j.text for j in i.find_all('td') if j.text != ''] for i in deetsoup.find(class_='tabset-moves-game sv-tabs-wrapper').find_all('tbody')[0]]


[['1', 'Twister', 'Dragon', '40', '100'],
 ['1', 'Vise Grip', 'Normal', '55', '100'],
 ['6', 'Bite', 'Dark', '60', '100'],
 ['12', 'Ancient Power', 'Rock', '60', '100'],
 ['18', 'Dragon Breath', 'Dragon', '60', '100'],
 ['24', 'Focus Energy', 'Normal', '—', '—'],
 ['30', 'Crunch', 'Dark', '80', '100'],
 ['36', 'Dragon Claw', 'Dragon', '80', '100'],
 ['42', 'Hammer Arm', 'Fighting', '100', '90'],
 ['48', 'Dragon Dance', 'Dragon', '—', '—'],
 ['54', 'Thrash', 'Normal', '120', '100'],
 ['60', 'Laser Focus', 'Normal', '—', '—'],
 ['66', 'Dragon Energy', 'Dragon', '150', '100'],
 ['72', 'Hyper Beam', 'Normal', '150', '90'],
 ['78', 'Explosion', 'Normal', '250', '100']]

In [35]:
# vitals-table returns 7 tables from the base url
#table0 = pokedex data
#table1 = training data - ev's and experience and frienship
#table2 = breeding data - egg groups
#table3 = base stats
#table4 = game flavor text
#table5 = locations per game
#table6 = names in other languages
data_tables = deetsoup.find_all('table', class_='vitals-table')
data_tables[0]

<table class="vitals-table">
<tbody>
<tr>
<th>National №</th>
<td><strong>895</strong></td>
</tr>
<tr>
<th>Type</th>
<td>
<a class="type-icon type-dragon" href="/type/dragon">Dragon</a> </td>
</tr>
<tr>
<th>Species</th>
<td>Dragon Orb Pokémon</td>
</tr>
<tr>
<th>Height</th>
<td>2.1 m (6′11″)</td>
</tr>
<tr>
<th>Weight</th>
<td>200.0 kg (440.9 lbs)</td>
</tr>
<tr>
<th>Abilities</th>
<td><span class="text-muted">1. <a href="/ability/dragons-maw" title="Signature ability of Regidrago. Powers up Dragon-type moves.">Dragon's Maw</a></span><br/></td>
</tr>
<tr>
<th>Local №</th>
<td>201 <small class="text-muted">(The Crown Tundra)</small><br/></td>
</tr>
</tbody>
</table>

In [36]:
table1_head = [i.text for i in data_tables[0].find_all('th')][:-1]
table2_head = [i.text for i in data_tables[1].find_all('th')]
table3_head = [i.text for i in data_tables[2].find_all('th')]

table1_head, table2_head, table3_head

(['National №', 'Type', 'Species', 'Height', 'Weight', 'Abilities'],
 ['EV yield', 'Catch rate', 'Base Friendship', 'Base Exp.', 'Growth Rate'],
 ['Egg Groups', 'Gender', 'Egg cycles'])

In [37]:
table1_data = [i.text for i in data_tables[0].find_all('td')]
table2_data = [i.text for i in data_tables[1].find_all('td')]
table3_data = [i.text for i in data_tables[2].find_all('td')]

table1_data, table2_data, table3_data

(['895',
  '\nDragon ',
  'Dragon Orb Pokémon',
  '2.1\xa0m (6′11″)',
  '200.0\xa0kg (440.9\xa0lbs)',
  "1. Dragon's Maw",
  '201 (The Crown Tundra)'],
 ['\n3 HP ',
  '\n3 (0.4% with PokéBall, full HP)\n',
  '\n35 (lower than normal)\n',
  '290',
  'Slow'],
 ['Undiscovered', 'Genderless', '120 (30,584–30,840 steps)\n'])

In [38]:
[i.get('href').split('/')[-1] for i in data_tables[0].find_all('a') if i.get('href').startswith('/ability/')]

['dragons-maw']

In [39]:
table2_data[0].replace('\n','')

'3 HP '

In [40]:
output = {'Name':[],
         'ID':[],
         'Type':[],
         'Species':[],
         'Height':[],
         'Weight':[],
         'Abilities':[],
         'EVs_Given':[],
         'Catch_Rate':[],
         'Base_Friendship':[],
         'Egg_Groups':[],
          'Gender_Spread':[],
          'Egg_Steps':[]
         }
to_change = {'Mr. Mime': 'mr-mime',
             'Mime Jr.': 'mime-jr',
             'Flabébé': 'flabebe',
             'Type: Null': 'type-null',
             'Tapu Koko': 'tapu-koko',
             'Tapu Lele': 'tapu-lele',
             'Tapu Bulu': 'tapu-bulu',
             'Tapu Fini': 'tapu-fini',
             "Sirfetch'd": 'sirfetchd',
             'Mr. Rime': 'mr-rime'}
pokemon_moves = {}

failed_images = []
failed_pokemon = []

try:
    os.mkdir('Artwork')
except:
    pass

for i in pokemon['Name']:
    if '♂' in i:
        i = i.replace('♂','-m')
    elif '♀' in i:
        i = i.replace('♀','-f')
    elif i in to_change.keys():
        i = to_change[i]
        
    time.sleep(1)
    base_url = 'https://pokemondb.net/pokedex/'
    url = base_url + i.lower()
    print(i)
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')
    data_tables = soup.find_all('table', class_='vitals-table')
    
    try:
        img_ref = soup.find('img').get('src')
        with open(f'Artwork/{i}.jpg', 'wb') as img:
            img.write(requests.get(img_ref).content)
    except:
        failed_images.append(i)
        
        
    try:
        moves = get_moves(soup)
        pokemon_moves[i] = moves
    except:
        pokemon_moves[i] = float('NaN')
        
        
    try:
        table1_data = [i.text for i in data_tables[0].find_all('td')][:-1]
        table2_data = [i.text for i in data_tables[1].find_all('td')][:-2]
        table3_data = [i.text for i in data_tables[2].find_all('td')]

        output['Name'].append(i)
        output['ID'].append(table1_data[0])
        output['Type'].append(table1_data[1])
        output['Species'].append(table1_data[2])
        output['Height'].append(table1_data[3].replace('\xa0',''))
        output['Weight'].append(table1_data[4].replace('\xa0',''))
        output['Abilities'].append([i.get('href').split('/')[-1] for i in data_tables[0].find_all('a') if i.get('href').startswith('/ability/')])
        output['EVs_Given'].append(table2_data[0].replace('/n',''))
        output['Catch_Rate'].append(table2_data[1].replace('/n',''))
        output['Base_Friendship'].append(table2_data[2].replace('/n',''))
        output['Egg_Groups'].append(table3_data[0])
        output['Gender_Spread'].append(table3_data[1])
        output['Egg_Steps'].append(table3_data[2])
    except:
        failed_pokemon.append(i)

Bulbasaur
Ivysaur
Venusaur
Charmander
Charmeleon
Charizard
Squirtle
Wartortle
Blastoise
Caterpie
Metapod
Butterfree
Weedle
Kakuna
Beedrill
Pidgey
Pidgeotto
Pidgeot
Rattata
Raticate
Spearow
Fearow
Ekans
Arbok
Pikachu
Raichu
Sandshrew
Sandslash
Nidoran-f
Nidorina
Nidoqueen
Nidoran-m
Nidorino
Nidoking
Clefairy
Clefable
Vulpix
Ninetales
Jigglypuff
Wigglytuff
Zubat
Golbat
Oddish
Gloom
Vileplume
Paras
Parasect
Venonat
Venomoth
Diglett
Dugtrio
Meowth
Persian
Psyduck
Golduck
Mankey
Primeape
Growlithe
Arcanine
Poliwag
Poliwhirl
Poliwrath
Abra
Kadabra
Alakazam
Machop
Machoke
Machamp
Bellsprout
Weepinbell
Victreebel
Tentacool
Tentacruel
Geodude
Graveler
Golem
Ponyta
Rapidash
Slowpoke
Slowbro
Magnemite
Magneton
Farfetch'd
Doduo
Dodrio
Seel
Dewgong
Grimer
Muk
Shellder
Cloyster
Gastly
Haunter
Gengar
Onix
Drowzee
Hypno
Krabby
Kingler
Voltorb
Electrode
Exeggcute
Exeggutor
Cubone
Marowak
Hitmonlee
Hitmonchan
Lickitung
Koffing
Weezing
Rhyhorn
Rhydon
Chansey
Tangela
Kangaskhan
Horsea
Seadra
Goldeen
Seaki

In [55]:
failed_pokemon, failed_images

([], [])

In [56]:
pokemon_details = pd.DataFrame(output)
pokemon_details.head()

,Name,ID,Type,Species,Height,Weight,Abilities,EVs_Given,Catch_Rate,Base_Friendship,Egg_Groups,Gender_Spread,Egg_Steps
0,Bulbasaur,001,\nGrass Poison,Seed Pokémon,0.7m (2′04″),6.9kg (15.2lbs),"[overgrow, chlorophyll]",\n1 Special Attack,"\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
1,Ivysaur,002,\nGrass Poison,Seed Pokémon,1.0m (3′03″),13.0kg (28.7lbs),"[overgrow, chlorophyll]","\n1 Special Attack, 1 Special Defense","\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
2,Venusaur,003,\nGrass Poison,Seed Pokémon,2.0m (6′07″),100.0kg (220.5lbs),"[overgrow, chlorophyll]","\n2 Special Attack, 1 Special Defense","\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Grass, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
3,Charmander,004,\nFire,Lizard Pokémon,0.6m (2′00″),8.5kg (18.7lbs),"[blaze, solar-power]",\n1 Speed,"\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Dragon, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"
4,Charmeleon,005,\nFire,Flame Pokémon,1.1m (3′07″),19.0kg (41.9lbs),"[blaze, solar-power]","\n1 Special Attack, 1 Speed","\n45 (5.9% with PokéBall, full HP)\n",\n50 (normal)\n,"Dragon, Monster","87.5% male, 12.5% female","20 (4,884–5,140 steps)\n"


In [57]:
for i in pokemon_details.columns:
    if i=="Abilities":
        pass
    else:
        pokemon_details[i] = pokemon_details[i].str.replace('\n','')
pokemon_details['Egg_Steps'] = pokemon_details['Egg_Steps'].str.replace('–','-').map(lambda x: re.findall('\((.+)\)',x)[0])

In [58]:
master = {'HP_EVs':[],
          'Atk_EVs':[],
          'Def_EVs':[],
          'Sp.Atk_EVs':[],
          'Sp.Def_EVs':[],
          'Spd_EVs':[]}

for x in [i.split(', ') for i in pokemon_details['EVs_Given']]:
    mon = {'hp':0,
          'attack':0,
          'defense':0,
          'specialattack':0,
          'specialdefense':0,
          'speed':0}
    
    for y in x:
        if y.endswith(' '):
            y = y[:-1]
        val = int(y[0])
        stat = ''.join(i.lower() for i in y if i.isalpha())
        mon[stat] += val
        
    master['HP_EVs'].append(mon['hp'])
    master['Atk_EVs'].append(mon['attack'])
    master['Def_EVs'].append(mon['defense'])
    master['Sp.Atk_EVs'].append(mon['specialattack'])
    master['Sp.Def_EVs'].append(mon['specialdefense'])
    master['Spd_EVs'].append(mon['speed'])
    

In [59]:
for k,v in master.items():
    pokemon_details[k] = v
pokemon_details= pokemon_details.drop(columns=['EVs_Given'])

In [60]:
pokemon_details['Egg_Steps'] = pokemon_details['Egg_Steps'].map(lambda x: re.findall('\-(.+)\s',x)[0].replace(',',''))
pokemon_details['Catch_Rate'] = pokemon_details['Catch_Rate'].apply(lambda x: x.split()[0])
pokemon_details['Base_Friendship'] = pokemon_details['Base_Friendship'].apply(lambda x: x.split(' ')[0])
pokemon_details = pokemon_details.drop(columns=['Abilities','Type'])

In [61]:
pokemon_details['Weight'] = pokemon_details['Weight'].map(lambda x: re.findall('\((.*)lbs\)', x)[0])
pokemon_details = pokemon_details.rename(columns={'Weight':'Weight_lbs'})

In [62]:
pokemon_details['Height'] = pokemon_details['Height'].map(lambda x: re.findall('\((.*)\)', x)[0])

In [63]:
genders = [(i.split(',')[0].replace('% male',''), i.split(',')[1].replace('% female','')) if i!='Genderless' else i for i in pokemon_details['Gender_Spread']]
pokemon_details['Male%'] = [i[0] if len(i)==2 else 'Genderless' for i in genders]
pokemon_details['Female%'] = [i[1] if len(i)==2 else 'Genderless' for i in genders]
pokemon_details = pokemon_details.drop(columns=['Gender_Spread'])

In [64]:
egg_groups = [[j for j in i.split() if not j.replace(',','').isnumeric()] for i in pokemon_details['Egg_Groups']]
pokemon_details['Egg_Group1'] = [i[0].replace(',','') for i in egg_groups]
pokemon_details['Egg_Group2'] = [i[1].replace(',','') if len(i)>1 else 'None' for i in egg_groups]
pokemon_details = pokemon_details.drop(columns=['Egg_Groups'])

In [65]:
pokemon_details.head()

,Name,ID,Species,Height,Weight_lbs,Catch_Rate,Base_Friendship,Egg_Steps,HP_EVs,Atk_EVs,Def_EVs,Sp.Atk_EVs,Sp.Def_EVs,Spd_EVs,Male%,Female%,Egg_Group1,Egg_Group2
0,Bulbasaur,001,Seed Pokémon,2′04″,15.2,45,50,5140,0,0,0,1,0,0,87.5,12.5,Grass,Monster
1,Ivysaur,002,Seed Pokémon,3′03″,28.7,45,50,5140,0,0,0,1,1,0,87.5,12.5,Grass,Monster
2,Venusaur,003,Seed Pokémon,6′07″,220.5,45,50,5140,0,0,0,2,1,0,87.5,12.5,Grass,Monster
3,Charmander,004,Lizard Pokémon,2′00″,18.7,45,50,5140,0,0,0,0,0,1,87.5,12.5,Dragon,Monster
4,Charmeleon,005,Flame Pokémon,3′07″,41.9,45,50,5140,0,0,0,1,0,1,87.5,12.5,Dragon,Monster


In [66]:
pokemon_details.to_csv('pokemon_details.csv', index=False)

In [471]:
pokemon_moves

{'Bulbasaur': {'Moves learnt by level up': [['1', 'Growl'],
   ['1', 'Tackle'],
   ['3', 'Vine Whip'],
   ['6', 'Growth'],
   ['9', 'Leech Seed'],
   ['12', 'Razor Leaf'],
   ['15', 'Poison Powder'],
   ['15', 'Sleep Powder'],
   ['18', 'Seed Bomb'],
   ['21', 'Take Down'],
   ['24', 'Sweet Scent'],
   ['27', 'Synthesis'],
   ['30', 'Worry Seed'],
   ['33', 'Double-Edge'],
   ['36', 'Solar Beam']],
  'Egg moves': ['Curse',
   'Ingrain',
   'Nature Power',
   'Petal Dance',
   'Skull Bash',
   'Toxic'],
  'Move Tutor moves': ['Grassy Glide'],
  'Moves learnt by TM': [['10', 'Magical Leaf'],
   ['11', 'Solar Beam'],
   ['17', 'Light Screen'],
   ['19', 'Safeguard'],
   ['21', 'Rest'],
   ['24', 'Snore'],
   ['25', 'Protect'],
   ['28', 'Giga Drain'],
   ['29', 'Charm'],
   ['31', 'Attract'],
   ['34', 'Sunny Day'],
   ['39', 'Facade'],
   ['41', 'Helping Hand'],
   ['46', 'Weather Ball'],
   ['50', 'Bullet Seed'],
   ['74', 'Venoshock'],
   ['76', 'Round'],
   ['88', 'Grassy Terrain'],
 

In [90]:
for i in pokemon_moves.keys():
    if not isinstance(pokemon_moves[i], dict):
        print(i)

Toxtricity
Indeedee
Urshifu
Calyrex


In [76]:
pokemon_moves['Toxtricity']

nan

# Broken, some categories are blank on the page to scrape

In [232]:
poke_moves = pd.DataFrame(pokemon_moves)
poke_moves.head()

,Bulbasaur,Ivysaur,Venusaur,Charmander,Charmeleon,Charizard,Squirtle,Wartortle,Blastoise,Caterpie,...,Zamazenta,Eternatus,Kubfu,Urshifu,Zarude,Regieleki,Regidrago,Glastrier,Spectrier,Calyrex
Moves learnt by level up,"[[1, Growl], [1, Tackle], [3, Vine Whip], [6, ...","[[1, Growl], [1, Growth], [1, Tackle], [1, Vin...","[[1, Growl], [1, Growth], [1, Petal Blizzard],...","[[1, Growl], [1, Scratch], [4, Ember], [8, Smo...","[[1, Ember], [1, Growl], [1, Scratch], [1, Smo...","[[1, Air Slash], [1, Dragon Claw], [1, Ember],...","[[1, Tackle], [1, Tail Whip], [3, Water Gun], ...","[[1, Tackle], [1, Tail Whip], [1, Water Gun], ...","[[1, Flash Cannon], [1, Tackle], [1, Tail Whip...","[[1, String Shot], [1, Tackle], [9, Bug Bite]]",...,"[[1, Bite], [1, Howl], [1, Metal Burst], [1, M...","[[1, Agility], [1, Confuse Ray], [1, Dragon Ta...","[[1, Leer], [1, Rock Smash], [4, Endure], [8, ...",NaN,"[[1, Bind], [1, Scratch], [6, Leer], [12, Vine...","[[1, Rapid Spin], [1, Thunder Shock], [6, Elec...","[[1, Twister], [1, Vise Grip], [6, Bite], [12,...","[[1, Tackle], [1, Tail Whip], [6, Double Kick]...","[[1, Tackle], [1, Tail Whip], [6, Double Kick]...",NaN
Egg moves,"[Curse, Ingrain, Nature Power, Petal Dance, Sk...","[Curse, Ingrain, Nature Power, Petal Dance, Sk...","[Curse, Ingrain, Nature Power, Petal Dance, Sk...","[Ancient Power, Belly Drum, Bite, Counter, Dra...","[Ancient Power, Belly Drum, Bite, Counter, Dra...","[Ancient Power, Belly Drum, Bite, Counter, Dra...","[Aqua Jet, Aqua Ring, Fake Out, Flail, Haze, L...","[Aqua Jet, Aqua Ring, Fake Out, Flail, Haze, L...","[Aqua Jet, Aqua Ring, Fake Out, Flail, Haze, L...",[82],...,[Coaching],[Meteor Beam],[Coaching],NaN,"[Grassy Glide, Lash Out]",[Rising Voltage],[Scale Shot],[Lash Out],[Lash Out],NaN
Move Tutor moves,[Grassy Glide],[Grassy Glide],"[Grassy Glide, Terrain Pulse]",NaN,NaN,"[Dual Wingbeat, Scale Shot, Scorching Sands]",[Flip Turn],[Flip Turn],"[Flip Turn, Terrain Pulse]",NaN,...,"[08, 09, 11, 15, 17, 18, 19, 21, 24, 25, 26, 3...","[06, 08, 09, 11, 16, 17, 18, 21, 24, 25, 26, 3...","[00, 01, 03, 04, 05, 15, 21, 24, 25, 26, 31, 3...",NaN,"[00, 01, 08, 09, 10, 11, 12, 15, 21, 22, 23, 2...","[08, 09, 14, 16, 17, 18, 20, 21, 24, 25, 33, 3...","[08, 09, 17, 18, 20, 21, 24, 25, 39, 66, 68, 7...","[08, 09, 21, 24, 25, 26, 27, 35, 39, 51, 53, 5...","[08, 09, 21, 24, 25, 26, 38, 39, 40, 53, 57, 5...",NaN
Moves learnt by TM,"[[10, Magical Leaf], [11, Solar Beam], [17, Li...","[[10, Magical Leaf], [11, Solar Beam], [17, Li...","[[08, Hyper Beam], [09, Giga Impact], [10, Mag...","[[00, Mega Punch], [01, Mega Kick], [03, Fire ...","[[00, Mega Punch], [01, Mega Kick], [03, Fire ...","[[00, Mega Punch], [01, Mega Kick], [03, Fire ...","[[00, Mega Punch], [01, Mega Kick], [04, Ice P...","[[00, Mega Punch], [01, Mega Kick], [04, Ice P...","[[00, Mega Punch], [01, Mega Kick], [04, Ice P...","[[1, String Shot], [1, Tackle]]",...,"[[12, Agility], [13, Focus Energy], [20, Subst...","[[02, Flamethrower], [12, Agility], [20, Subst...","[[01, Body Slam], [07, Low Kick], [13, Focus E...",NaN,"[[01, Body Slam], [07, Low Kick], [20, Substit...","[[01, Body Slam], [08, Thunderbolt], [09, Thun...","[[01, Body Slam], [13, Focus Energy], [20, Sub...","[[00, Swords Dance], [01, Body Slam], [05, Ice...","[[01, Body Slam], [12, Agility], [20, Substitu...",NaN
Moves learnt by TR,"[[00, Swords Dance], [01, Body Slam], [17, Amn...","[[00, Swords Dance], [01, Body Slam], [17, Amn...","[[00, Swords Dance], [01, Body Slam], [10, Ear...","[[00, Swords Dance], [01, Body Slam], [02, Fla...","[[00, Swords Dance], [01, Body Slam], [02, Fla...","[[00, Swords Dance], [01, Body Slam], [02, Fla...","[[01, Body Slam], [03, Hydro Pump], [04, Surf]...","[[01, Body Slam], [03, Hydro Pump], [04, Surf]...","[[01, Body Slam], [03, Hydro Pump], [04, Surf]...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [224]:
poke_moves.swapaxes(axis1='index', axis2='columns').sample(10)

,Moves learnt by level up,Egg moves,Move Tutor moves,Moves learnt by TM,Moves learnt by TR,Moves learnt on evolution,Transfer-only moves,Pre-evolution moves
Regice,"[[1, Charge Beam], [1, Icy Wind], [6, Bulldoze...","[00, 01, 04, 05, 08, 09, 14, 19, 20, 21, 22, 2...",NaN,"[[01, Body Slam], [05, Ice Beam], [06, Blizzar...","[[1, Bulldoze], [1, Charge Beam], [1, Explosio...",NaN,NaN,NaN
Arceus,"[[1, Cosmic Power], [1, Natural Gift], [1, Pun...","[Aqua Tail, Defog, Draco Meteor, Dragon Pulse,...","[01, 02, 03, 04, 05, 06, 07, 10, 11, 13, 14, 1...","[[Avalanche, Ice], [Brine, Water], [Bullet See...",NaN,NaN,"[1, 1, 1, 1, 10, 20, 30, 40, 50, 60, 70, 80, 9...",NaN
Hatenna,"[[1, Confusion], [1, Play Nice], [5, Life Dew]...","[After You, Aromatic Mist, Nuzzle, Quash]",[Expanding Force],"[[10, Magical Leaf], [14, Thunder Wave], [17, ...","[[11, Psychic], [20, Substitute], [25, Psyshoc...",NaN,NaN,NaN
Florges,"[[1, Aromatherapy], [1, Disarming Voice], [1, ...","[Camouflage, Captivate, Copycat, Tearful Look]","[After You, Ally Switch, Covet, Defog, Endeavo...","[[04, Calm Mind], [06, Toxic], [10, Hidden Pow...",NaN,NaN,"[Flash, Secret Power]","[Fairy Wind, Razor Leaf, Solar Beam, Tackle, V..."
Slowking,"[[1, Curse], [1, Growl], [1, Nasty Plot], [1, ...",[Eerie Spell],"[Belch, Belly Drum, Block, Stomp]","[[Expanding Force, Psychic]]","[[00, Mega Punch], [01, Mega Kick], [02, Pay D...","[1, 1, 1, 1, 1, 1, 1, 1, 9, 12, 15, 18, 21, 24...",NaN,NaN
Spinarak,"[[1, Constrict], [1, Poison Sting], [1, String...","[Baton Pass, Disable, Electroweb, Lunge, Megah...","[Bounce, Bug Bite, Electroweb, Foul Play, Giga...","[[06, Toxic], [09, Venoshock], [10, Hidden Pow...",NaN,NaN,"[Screech, Curse, Body Slam, Double-Edge, Mimic...",NaN
Timburr,"[[1, Leer], [1, Pound], [4, Low Kick], [8, Roc...","[Counter, Defog, Detect, Mach Punch, Power-Up ...",[Coaching],"[[00, Mega Punch], [01, Mega Kick], [03, Fire ...","[[07, Low Kick], [13, Focus Energy], [20, Subs...",NaN,NaN,NaN
Dreepy,"[[1, Astonish], [1, Bite], [1, Infestation], [...","[Confuse Ray, Curse, Disable, Double Team, Dra...",NaN,"[[14, Thunder Wave], [21, Rest], [24, Snore], ...","[[20, Substitute], [26, Endure], [27, Sleep Ta...",NaN,NaN,NaN
Caterpie,"[[1, String Shot], [1, Tackle], [9, Bug Bite]]",[82],NaN,"[[1, String Shot], [1, Tackle]]",NaN,NaN,NaN,NaN
Drizzile,"[[1, Bind], [1, Growl], [1, Pound], [1, Water ...","[Aqua Jet, Aqua Ring, Double Team, Fell Stinge...",NaN,"[[17, Light Screen], [18, Reflect], [19, Safeg...","[[04, Surf], [20, Substitute], [26, Endure], [...",NaN,NaN,NaN


In [233]:
poke_moves.to_csv('shitty_move_list.csv', index=False)